In [2]:
import os

import altair as alt
import pandas as pd
import numpy as np
import scipy
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, f1_score, accuracy_score

import mgitools.os_helpers as os_helpers

In [3]:
DATA_DIR = './results/01272021_harmonized_v2/'

In [48]:
# list stats

In [49]:
fps = sorted(os_helpers.listfiles('./data/01272021_harmonized/teir_1/', regex='.h5ad$'))
fps = [x for x in fps if 'train' not in x and 'val' not in x and 'gene' not in x and 'motif' not in x and 'peak' not in x]
fps

['./data/01272021_harmonized/teir_1/scRNAseq/brca.h5ad',
 './data/01272021_harmonized/teir_1/scRNAseq/cesc.h5ad',
 './data/01272021_harmonized/teir_1/scRNAseq/hnscc.h5ad',
 './data/01272021_harmonized/teir_1/scRNAseq/melanoma.h5ad',
 './data/01272021_harmonized/teir_1/scRNAseq/myeloma.h5ad',
 './data/01272021_harmonized/teir_1/scRNAseq/pdac.h5ad',
 './data/01272021_harmonized/teir_1/snATACseq/brca.h5ad',
 './data/01272021_harmonized/teir_1/snATACseq/ccrcc.h5ad',
 './data/01272021_harmonized/teir_1/snATACseq/gbm.h5ad',
 './data/01272021_harmonized/teir_1/snRNAseq/brca.h5ad',
 './data/01272021_harmonized/teir_1/snRNAseq/ccrcc.h5ad',
 './data/01272021_harmonized/teir_1/snRNAseq/gbm.h5ad']

In [ ]:
data = []
for fp in fps:
    print(fp)
    dtype = fp.split('/')[-2]
    disease = fp.split('/')[-1].split('.')[0]
    a = sc.read_h5ad(fp)
    n_cells = a.shape[0]
    if 'pbmc' not in fp:
        
        if dtype == 'snRNAseq' and disease == 'ccrcc':
            n_samples = len(set(a.obs['orig.ident_x']))
        else:
            n_samples = len(set(a.obs['orig.ident']))

        data.append([dtype, disease, n_cells, n_samples])
df = pd.DataFrame(data=data, columns=['datatype', 'disease', 'num cells', 'num samples'])
df
    
    

./data/01272021_harmonized/teir_1/scRNAseq/brca.h5ad
./data/01272021_harmonized/teir_1/scRNAseq/cesc.h5ad
./data/01272021_harmonized/teir_1/scRNAseq/hnscc.h5ad


In [ ]:
df.loc[1, 'num samples'] = 9
df.loc[2, 'num samples'] = 33
df.loc[3, 'num samples'] = 8
df.loc[7, 'num samples'] = 9
df.loc[8, 'num samples'] = 10

In [96]:
df

,datatype,disease,num cells,num samples
0,scRNAseq,brca,98564,30
1,scRNAseq,cesc,31745,9
2,scRNAseq,hnscc,227195,33
3,scRNAseq,melanoma,52367,8
4,scRNAseq,myeloma,92446,31
5,scRNAseq,pdac,177089,58
6,snATACseq,brca,70679,12
7,snATACseq,ccrcc,79851,9
8,snATACseq,gbm,65782,10
9,snRNAseq,brca,109002,12


In [92]:
a = sc.read_h5ad('./data/01272021_harmonized/teir_1/snATACseq/gbm.h5ad')
a

AnnData object with n_obs × n_vars = 65782 × 19891
    obs: 'orig.ident', 'nCount_peaks', 'nFeature_peaks', 'total', 'duplicate', 'chimeric', 'unmapped', 'lowmapq', 'mitochondrial', 'passed_filters', 'cell_id', 'is__cell_barcode', 'TSS_fragments', 'DNase_sensitive_region_fragments', 'enhancer_region_fragments', 'promoter_region_fragments', 'on_target_fragments', 'blacklist_region_fragments', 'peak_region_fragments', 'peak_region_cutsites', 'nucleosome_signal', 'nucleosome_percentile', 'TSS.enrichment', 'TSS.percentile', 'pct_reads_in_peaks', 'blacklist_ratio', 'high.tss', 'nucleosome_group', 'peaks_snn_res.0.8', 'seurat_clusters', 'nCount_RNA', 'nFeature_RNA', 'predicted.id', 'prediction.score.Tumor', 'prediction.score.Neurons', 'prediction.score.Microglia', 'prediction.score.Doublets', 'prediction.score.Oligodendrocytes', 'prediction.score.T.cells', 'prediction.score.Fibroblasts', 'prediction.score.B.cells.Plasma', 'prediction.score.Maybe.OPC', 'prediction.score.Endothelial.cells', 'p

In [93]:
a.obs

,orig.ident,nCount_peaks,nFeature_peaks,total,duplicate,chimeric,unmapped,lowmapq,mitochondrial,passed_filters,...,prediction.score.Fibroblasts,prediction.score.B.cells.Plasma,prediction.score.Maybe.OPC,prediction.score.Endothelial.cells,prediction.score.max,nCount_peaksinters,nFeature_peaksinters,dataset,prediction.score.Monocytes,cell_type
C3L-02705_CPT0189650015_2020-06-16_GCGAGAACATGCCCTT-1,ATAC,11593.0,4952,19264,5375,491,244,1706,65,11383,...,0.0,0.0,0.000000,0.000000,0.966328,7049.0,5941,C3L-02705_CPT0189650015_2020-06-16,NaN,Malignant
C3L-02705_CPT0189650015_2020-06-16_CTCAGCTGTTATGCAC-1,ATAC,7530.0,3441,13929,3733,362,144,1324,15,8351,...,0.0,0.0,0.000000,0.000000,0.851200,4524.0,3989,C3L-02705_CPT0189650015_2020-06-16,NaN,Microglia
C3L-02705_CPT0189650015_2020-06-16_ACCCAAAAGATCTAAG-1,ATAC,3645.0,1761,7842,1945,150,118,632,23,4974,...,0.0,0.0,0.000000,0.000000,1.000000,2472.0,2275,C3L-02705_CPT0189650015_2020-06-16,NaN,Malignant
C3L-02705_CPT0189650015_2020-06-16_AACTGTGCACGCGCAT-1,ATAC,5921.0,2788,10175,2807,253,164,983,24,5944,...,0.0,0.0,0.000000,0.000000,0.997391,3488.0,3150,C3L-02705_CPT0189650015_2020-06-16,NaN,Microglia
C3L-02705_CPT0189650015_2020-06-16_TTGCTTAGTCCGTGCA-1,ATAC,2092.0,1049,5986,1388,70,119,597,1,3811,...,0.0,0.0,0.000000,0.000000,0.932418,1527.0,1422,C3L-02705_CPT0189650015_2020-06-16,NaN,Malignant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C3N-03186_CPT0206880004_2020-06-22_AAACTGCTCCTTCGAC-1,ATAC,9837.0,4306,34007,8711,390,330,3219,222,21135,...,0.0,0.0,0.000000,0.000000,1.000000,6454.0,5412,C3N-03186_CPT0206880004_2020-06-22,NaN,Malignant
C3N-03186_CPT0206880004_2020-06-22_ATAGTCGAGACGTCAG-1,ATAC,2448.0,1166,6307,1694,71,75,731,22,3714,...,0.0,0.0,0.000000,0.000000,1.000000,1750.0,1597,C3N-03186_CPT0206880004_2020-06-22,NaN,Microglia
C3N-03186_CPT0206880004_2020-06-22_CTCACCAGTAACAATG-1,ATAC,2383.0,1147,9788,2440,110,122,1079,754,5283,...,0.0,0.0,0.000197,0.004569,0.550301,1519.0,1416,C3N-03186_CPT0206880004_2020-06-22,NaN,Microglia
C3N-03186_CPT0206880004_2020-06-22_CCCAGAGCAATGGTCT-1,ATAC,3730.0,1753,14409,3574,131,120,1234,306,9044,...,0.0,0.0,0.000000,0.000000,1.000000,2947.0,2643,C3N-03186_CPT0206880004_2020-06-22,NaN,Microglia


In [94]:
set(a.obs['orig.ident'])
{'_'.join(x.split('_')[:-1]) for x in a.obs.index}

{'C3L-02705_CPT0189650015_2020-06-16',
 'C3N-01334_CPT0125220004_2020-06-18',
 'C3N-01518_CPT0167640014_2020-07-22',
 'C3N-01816_CPT0168080014_2020-06-08',
 'C3N-01818_CPT0168270014_2020-07-23',
 'C3N-02186_CPT0168720014_2020-07-27',
 'C3N-02188_CPT0168830014_2020-06-03',
 'C3N-02769_CPT0205570014_2020-06-15',
 'C3N-02784_CPT0206000015_2020-06-10',
 'C3N-03186_CPT0206880004_2020-06-22'}

In [87]:
len(set(a.obs['ident']))

8

In [4]:
fps = sorted(os_helpers.listfiles(DATA_DIR, regex='.tsv$'))
fps = [fp for fp in fps if not ('snATACseq' in fp and 'gene_activity' not in fp)]
fps = [fp for fp in fps if not ('train/' in fp or 'val/' in fp)]
# fps = [x for x in fps if 'seurat' not in x]
fps

['./results/01272021_harmonized_v2/scRNAseq/brca/SingleCellNet.tsv',
 './results/01272021_harmonized_v2/scRNAseq/brca/actinn.tsv',
 './results/01272021_harmonized_v2/scRNAseq/brca/pollock.tsv',
 './results/01272021_harmonized_v2/scRNAseq/cesc/SingleCellNet.tsv',
 './results/01272021_harmonized_v2/scRNAseq/cesc/actinn.tsv',
 './results/01272021_harmonized_v2/scRNAseq/cesc/pollock.tsv',
 './results/01272021_harmonized_v2/scRNAseq/hnscc/SingleCellNet.tsv',
 './results/01272021_harmonized_v2/scRNAseq/hnscc/actinn.tsv',
 './results/01272021_harmonized_v2/scRNAseq/hnscc/pollock.tsv',
 './results/01272021_harmonized_v2/scRNAseq/melanoma/SingleCellNet.tsv',
 './results/01272021_harmonized_v2/scRNAseq/melanoma/pollock.tsv',
 './results/01272021_harmonized_v2/scRNAseq/myeloma/SingleCellNet.tsv',
 './results/01272021_harmonized_v2/scRNAseq/myeloma/pollock.tsv',
 './results/01272021_harmonized_v2/scRNAseq/pdac/SingleCellNet.tsv',
 './results/01272021_harmonized_v2/scRNAseq/pdac/actinn.tsv',
 './re

In [5]:
pred_map = {}
for fp in fps:
    dtype = fp.split('/')[-3]
    disease = fp.split('/')[-2].upper()
    method = fp.split('/')[-1].replace('.tsv', '')
    
    if dtype == 'snATACseq': disease = disease.replace('_GENE_ACTIVITY', '')
    
    if 'actinn' in method: method = method.upper()
#     if 'scanpy' in method: method = 'Scanpy Ingest'
    if 'pollock' in method: method = 'Pollock'
    try:
        df = pd.read_csv(fp, sep='\t')
        df.columns = ['cell_id', 'groundtruth', 'predicted', 'probability']
        if dtype not in pred_map: pred_map[dtype] = {}
        if disease not in pred_map[dtype]: pred_map[dtype][disease] = {}
        pred_map[dtype][disease][method] = df
    except:
        print('failed: ', fp)


In [6]:
pred_map['snATACseq'].keys()

dict_keys(['BRCA', 'CCRCC', 'GBM'])

In [7]:
# pred_map['scRNAseq'].pop('PBMC')

In [8]:
def get_performance_df(d):
    diseases = sorted(d.keys())
    methods = sorted(d[diseases[0]].keys())
    ds, scores, ms = [], [], []
    for disease in diseases:
        for method in methods:
            if method in d[disease]:
                df = d[disease][method]
                scores.append(f1_score(df['groundtruth'], df['predicted'], average='weighted'))
                ds.append(disease)
                ms.append(method)
    df = pd.DataFrame.from_dict({
        'disease': ds,
        'method': ms,
        'f1-score': scores
    })
    return df

In [9]:
df = get_performance_df(pred_map['snATACseq'])
df

,disease,method,f1-score
0,BRCA,ACTINN,0.936859
1,BRCA,Pollock,0.884249
2,BRCA,SingleCellNet,0.881482
3,CCRCC,ACTINN,0.938560
4,CCRCC,Pollock,0.887064
5,CCRCC,SingleCellNet,0.879497
6,GBM,ACTINN,0.950578
7,GBM,Pollock,0.900447
8,GBM,SingleCellNet,0.891949


In [10]:
get_performance_df(pred_map['snRNAseq'])

,disease,method,f1-score
0,BRCA,ACTINN,0.899281
1,BRCA,Pollock,0.882375
2,BRCA,SingleCellNet,0.898215
3,CCRCC,ACTINN,0.943742
4,CCRCC,Pollock,0.914359
5,CCRCC,SingleCellNet,0.909850
6,GBM,ACTINN,0.979644
7,GBM,Pollock,0.954746
8,GBM,SingleCellNet,0.975218


In [11]:
get_performance_df(pred_map['scRNAseq'])

,disease,method,f1-score
0,BRCA,ACTINN,0.888478
1,BRCA,Pollock,0.850659
2,BRCA,SingleCellNet,0.883962
3,CESC,ACTINN,0.970190
4,CESC,Pollock,0.951904
5,CESC,SingleCellNet,0.972933
6,HNSCC,ACTINN,0.881691
7,HNSCC,Pollock,0.860325
8,HNSCC,SingleCellNet,0.870945
9,MELANOMA,Pollock,0.924106


In [12]:
# dtype_order = ['scRNAseq', 'snRNAseq', 'snATACseq']
dtype_order = ['scRNAseq', 'snRNAseq', 'snATACseq']

In [13]:
def make_tick_chart(df, title='', show_axis=True):
    chart = alt.Chart(df, title=title)
    chart = chart.mark_tick(thickness=2,).encode(
        y=alt.Y(f'disease:N', axis=alt.Axis(title=None)),
        x=alt.X(f'f1-score:Q', scale=alt.Scale(zero=True), axis=alt.Axis(grid=True, labels=show_axis,
                                                                        title='F1-score' if show_axis else None)),
        color=alt.Color('method:N', sort=['Pollock'])
    )
    return chart

def make_point_chart(df, show_axis=True):
    chart = alt.Chart(df)
    chart = chart.mark_point().encode(
        y=alt.Y(f'disease:N', axis=alt.Axis(title=None)),
        x=alt.X(f'f1-score:Q', scale=alt.Scale(zero=True), axis=alt.Axis(grid=True, labels=show_axis,
                                                                        title='F1-score' if show_axis else None)),
        color=alt.Color('method:N', sort=['Pollock'])
    )
    return chart

def make_chart(df, title='', show_axis=True, color='red'):
    chart = alt.Chart(df, title=title)
    chart = chart.mark_point(color=color).encode(
        y=alt.Y(f'disease:N', axis=alt.Axis(title=None)),
        x=alt.X(f'f1-score:Q', scale=alt.Scale(zero=True), axis=alt.Axis(grid=True, labels=show_axis,
                                                                        title='F1-score' if show_axis else None)),
        shape=alt.Shape('method:N', sort=['Pollock'])
#         color=alt.Color('method:N', sort=['Pollock'])
    )
    return chart

In [14]:
combined = None
for i, dtype in enumerate(dtype_order):
    d = pred_map[dtype]
    df = get_performance_df(d)
#     df['disease'] = [x.upper() for x in df['disease']]
    print(df.shape)
    chart = make_point_chart(df, show_axis=True if i==2 else False) + make_tick_chart(df, title=dtype, show_axis=True if i==2 else False)
#     chart = make_chart(df, title=dtype, show_axis=True if i==2 else False, color=sns.color_palette().as_hex()[i])
    if combined is None:
        combined = chart
    else:
        combined = alt.vconcat(combined, chart)
#         combined |= chart
combined.resolve_scale(x='shared', )

(16, 3)
(9, 3)
(9, 3)


alt.VConcatChart(...)

In [15]:


combined = None
for dtype in dtype_order:
    d = pred_map[dtype]
    df = get_performance_df(d)
    df['disease'] = [x.upper() for x in df['disease']]
    print(df.shape)
    chart = make_point_chart(df) + make_tick_chart(df, title=dtype)
    if combined is None:
        combined = chart
    else:
        combined |= chart
combined.resolve_scale(y='shared')

(16, 3)
(9, 3)
(9, 3)


alt.HConcatChart(...)

In [16]:
pred_map['scRNAseq']['PDAC']['Pollock']

,cell_id,groundtruth,predicted,probability
0,HT056P_S1PA_AAACGAAGTTTACGTG-1,Epithelial,Epithelial,0.89
1,HT056P_S1PA_AAAGTGACATCAGCTA-1,Monocyte,Epithelial,0.35
2,HT056P_S1PA_AACAACCGTAGCGATG-1,Tuft,Tuft,0.74
3,HT056P_S1PA_AACAAGACACATTCGA-1,Tuft,Tuft,0.60
4,HT056P_S1PA_AACACACCAAGGCGTA-1,Plasma,Plasma,1.00
...,...,...,...,...
7818,HT168P1_S1H4_TGCACGGGTCAATCTG-1,Endothelial,Endothelial,0.95
7819,HT168P1_S1H4_TGGATGTAGTCTGTAC-1,Fibroblast,Fibroblast,0.76
7820,HT168P1_S1H4_TTCGCTGGTCTTCATT-1,Endothelial,Endothelial,0.27
7821,HT168P1_S1H4_TTGCATTCATAGAGGC-1,Monocyte,Monocyte,0.50


In [17]:
def build_f1_matrix(d):
    
    cell_type_labels = {c for df in d.values() for c in df['groundtruth']}
#     cell_type_labels.update({c for df in d.values() for c in df['predicted']})
    cell_types = sorted(cell_type_labels)

    data = []
    labels = []
    for label, df in d.items():
        labels.append(label)
        report = classification_report(df['groundtruth'], df['predicted'], labels=cell_types, output_dict=True)
        data.append([report[c]['f1-score'] for c in cell_types])
    df = pd.DataFrame(data=data, columns=cell_types, index=labels)
    return df
    

In [18]:
df = build_f1_matrix(pred_map['scRNAseq']['BRCA'])
df['method'] = df.index.to_list()
df = df.melt(id_vars=['method'])
df.columns = ['method', 'cell type', 'f1-score']
df

,method,cell type,f1-score
0,SingleCellNet,B cell,0.989076
1,ACTINN,B cell,0.994012
2,Pollock,B cell,0.974975
3,SingleCellNet,CD4 T cell,0.769691
4,ACTINN,CD4 T cell,0.773512
5,Pollock,CD4 T cell,0.727273
6,SingleCellNet,CD8 T cell,0.773176
7,ACTINN,CD8 T cell,0.754404
8,Pollock,CD8 T cell,0.665990
9,SingleCellNet,Dendritic,0.932331


In [19]:
def get_cell_f1_heatmap(df, title='', keep_x=True, keep_y=True, y_title=''):
    x_kwargs, y_kwargs = {}, {}
    if not keep_x: x_kwargs['axis'] = None
    if not keep_y: y_kwargs['axis'] = None
    return alt.Chart(df, title=title).mark_rect().encode(
        x=alt.X('cell type:N', title='', **x_kwargs),
        y=alt.Y('method:N', title=y_title, sort=['Pollock'], **y_kwargs),
        color=alt.Color('f1-score:Q',)
    )

In [20]:
# for dtype, d in pred_map.items():
d = pred_map['scRNAseq']
combined = None
for i, dtype in enumerate(dtype_order):
    d = pred_map[dtype]
    row = None
    for disease, m in d.items():
        df = build_f1_matrix(m)
        df['method'] = df.index.to_list()
        df = df.melt(id_vars=['method'])
        df.columns = ['method', 'cell type', 'f1-score']
        keep_x = True if i == len(pred_map) - 1 else False
        if row is None:
            row = get_cell_f1_heatmap(df, y_title=dtype, title=disease)
        else:
            row = alt.hconcat(row, get_cell_f1_heatmap(df, keep_y=False, title=disease), spacing=2)
            
    if combined is None:
        combined = row
    else:
        combined = alt.vconcat(combined, row, spacing=-35)
combined

/home/estorrs/miniconda3/envs/pollock_dev/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


alt.VConcatChart(...)

In [21]:
get_cell_f1_heatmap(df)

alt.Chart(...)

data availability dot plot

In [22]:
import scanpy as sc
fps = sorted(os_helpers.listfiles('/home/estorrs/pollock/benchmarking/data/01272021_harmonized/teir_1/', regex='.h5ad$'))
size_map = {fp.split('/')[-2]:{} for fp in fps}
for fp in fps:
    if '/_train.h5ad' not in fp and '/_val.h5ad' not in fp:
        dtype = fp.split('/')[-2]
        disease = fp.split('/')[-1].replace('.h5ad', '')
        size_map[dtype][disease] = sc.read_h5ad(fp).shape[0]
size_map

{'scRNAseq': {'brca': 98564,
  'brca_train': 6105,
  'brca_val': 5748,
  'cesc': 31745,
  'cesc_train': 4661,
  'cesc_val': 4276,
  'hnscc': 227195,
  'hnscc_train': 5287,
  'hnscc_val': 5201,
  'melanoma': 52367,
  'melanoma_train': 4218,
  'melanoma_val': 3517,
  'myeloma': 92446,
  'myeloma_train': 3617,
  'myeloma_val': 3312,
  'pdac': 177089,
  'pdac_train': 7940,
  'pdac_val': 7823},
 'snATACseq': {'brca': 70679,
  'brca_gene_activity': 90757,
  'brca_gene_activity_train': 3576,
  'brca_gene_activity_val': 3519,
  'brca_motif': 90757,
  'brca_motif_train': 3576,
  'brca_motif_val': 3519,
  'brca_peaks': 90757,
  'brca_peaks_train': 3576,
  'brca_peaks_val': 3519,
  'brca_train': 4836,
  'brca_val': 4656,
  'ccrcc': 79851,
  'ccrcc_gene_activity': 79852,
  'ccrcc_gene_activity_train': 3000,
  'ccrcc_gene_activity_val': 3000,
  'ccrcc_motif': 79852,
  'ccrcc_motif_train': 3000,
  'ccrcc_motif_val': 3000,
  'ccrcc_peaks': 79852,
  'ccrcc_peaks_train': 3000,
  'ccrcc_peaks_val': 3000

In [23]:
# sm = {k:{c.upper():size for c, size in v.items() if 'train' not in c and 'val' not in c} for k, v in size_map.items()}
# sm

In [24]:
# dtype_order
# disease_order = sorted({d for k, ds in pred_map.items() for d in ds.keys()})

# data = []
# for dtype in dtype_order:
#     for disease in disease_order:
#         if dtype in pred_map and disease in pred_map[dtype]:
#             data.append([dtype, disease])
            
# data = [[dtype, disease, sm[dtype][disease]] for dtype, d in size_map.items() for disease in d.keys()
#        if not (dtype=='snATACseq' and disease in ['CCRCC', 'CCRCC_MOTIF', 'GBM', 'GBM_MOTIF', 'BRCA', 'BRCA_MOTIF'])]
data = []
for dtype, d1 in size_map.items():
    for disease, count in d1.items():
        if 'motif' not in disease and 'gene' not in disease and 'peak' not in disease:
            if 'train' not in disease and 'val' not in disease and 'pbmc' not in disease:
                data.append([dtype, disease, count])

df = pd.DataFrame(data=data, columns=['data type', 'disease', 'num cells'])
df

,data type,disease,num cells
0,scRNAseq,brca,98564
1,scRNAseq,cesc,31745
2,scRNAseq,hnscc,227195
3,scRNAseq,melanoma,52367
4,scRNAseq,myeloma,92446
5,scRNAseq,pdac,177089
6,snATACseq,brca,70679
7,snATACseq,ccrcc,79851
8,snATACseq,gbm,65782
9,snRNAseq,brca,109002


In [25]:
chart = alt.Chart(df).mark_circle().encode(
    x=alt.X('disease:O'),
    y=alt.Y('data type:O'),
    color=alt.Color('num cells:Q'),
    size='num cells:Q'
)
chart

alt.Chart(...)

In [26]:
import scanpy as sc

In [27]:
fps = sorted(os_helpers.listfiles('/home/estorrs/pollock/benchmarking/data/01272021_harmonized/teir_1/', regex='.h5ad$'))
size_map_specific = {fp.split('/')[-2]:{} for fp in fps}
for fp in fps:
    if '/_train.h5ad' not in fp and '/_val.h5ad' not in fp:
        dtype = fp.split('/')[-2]
        disease = fp.split('/')[-1].replace('.h5ad', '')
        a = sc.read_h5ad(fp)
        cell_types = sorted(set(a.obs['cell_type']))
        d = {k:[] for k in cell_types}
        for c in cell_types:
            d[c] = a[a.obs['cell_type']==c].shape[0]
        size_map_specific[dtype][disease] = d
size_map_specific

/home/estorrs/miniconda3/envs/pollock_dev/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):


{'scRNAseq': {'brca': {'B cell': 2972,
   'CD4 T cell': 11184,
   'CD8 T cell': 11649,
   'Dendritic': 592,
   'Endothelial': 6188,
   'Erythrocyte': 515,
   'Fibroblast': 14769,
   'Malignant': 27884,
   'Mast': 746,
   'Monocyte': 9824,
   'NK': 2958,
   'Plasma': 4879,
   'Treg': 4404},
  'brca_train': {'B cell': 500,
   'CD4 T cell': 500,
   'CD8 T cell': 500,
   'Dendritic': 320,
   'Endothelial': 500,
   'Erythrocyte': 285,
   'Fibroblast': 500,
   'Malignant': 500,
   'Mast': 500,
   'Monocyte': 500,
   'NK': 500,
   'Plasma': 500,
   'Treg': 500},
  'brca_val': {'B cell': 500,
   'CD4 T cell': 500,
   'CD8 T cell': 500,
   'Dendritic': 272,
   'Endothelial': 500,
   'Erythrocyte': 230,
   'Fibroblast': 500,
   'Malignant': 500,
   'Mast': 246,
   'Monocyte': 500,
   'NK': 500,
   'Plasma': 500,
   'Treg': 500},
  'cesc': {'CD4 T cell': 1118,
   'CD8 T cell': 2523,
   'Endothelial': 732,
   'Epithelial': 3815,
   'Erythrocyte': 67,
   'Fibroblast': 2231,
   'Malignant': 16485,
 

In [28]:
cell_types = sorted({c for k, d in size_map_specific.items() for dis, m in d.items() for c, _ in m.items()})
cell_types

['Acinar',
 'Adipocyte',
 'B cell',
 'CD4 T cell',
 'CD8 T cell',
 'Dendritic',
 'Endothelial',
 'Epithelial',
 'Erythrocyte',
 'Fibroblast',
 'Islet',
 'Malignant',
 'Mast',
 'Microglia',
 'Monocyte',
 'NK',
 'Neuron',
 'Oligodendrocytes',
 'Plasma',
 'Platlete',
 'T cells',
 'Treg',
 'Tuft']

In [29]:
data = []
for dtype, d1 in size_map_specific.items():
    for disease, d2 in d1.items():
        for c in cell_types:
            count = d2.get(c, 0)
            if 'val' not in disease and 'train' not in disease:
                if 'motif' not in disease and 'peak' not in disease and 'gene' not in disease:
                    data.append([dtype, disease, c, count])
df = pd.DataFrame(data=data, columns=['Data type', 'Disease', 'Cell type', 'Number of cells'])
df

,Data type,Disease,Cell type,Number of cells
0,scRNAseq,brca,Acinar,0
1,scRNAseq,brca,Adipocyte,0
2,scRNAseq,brca,B cell,2972
3,scRNAseq,brca,CD4 T cell,11184
4,scRNAseq,brca,CD8 T cell,11649
...,...,...,...,...
271,snRNAseq,gbm,Plasma,0
272,snRNAseq,gbm,Platlete,0
273,snRNAseq,gbm,T cells,10320
274,snRNAseq,gbm,Treg,0


In [30]:
chart = alt.Chart(df).mark_rect().encode(
        x=alt.X('cell type:N'),
        y=alt.Y('disease:N'))
chart

alt.Chart(...)

In [41]:
for i, row in df.iterrows():
    if row['Number of cells'] > 0:
        print(row['Data type'], row['Disease'], row['Cell type'], row['Number of cells'])

scRNAseq brca B cell 2972
scRNAseq brca CD4 T cell 11184
scRNAseq brca CD8 T cell 11649
scRNAseq brca Dendritic 592
scRNAseq brca Endothelial 6188
scRNAseq brca Erythrocyte 515
scRNAseq brca Fibroblast 14769
scRNAseq brca Malignant 27884
scRNAseq brca Mast 746
scRNAseq brca Monocyte 9824
scRNAseq brca NK 2958
scRNAseq brca Plasma 4879
scRNAseq brca Treg 4404
scRNAseq cesc CD4 T cell 1118
scRNAseq cesc CD8 T cell 2523
scRNAseq cesc Endothelial 732
scRNAseq cesc Epithelial 3815
scRNAseq cesc Erythrocyte 67
scRNAseq cesc Fibroblast 2231
scRNAseq cesc Malignant 16485
scRNAseq cesc Mast 138
scRNAseq cesc Monocyte 2149
scRNAseq cesc NK 1352
scRNAseq cesc Plasma 1135
scRNAseq hnscc B cell 19300
scRNAseq hnscc CD4 T cell 31826
scRNAseq hnscc CD8 T cell 15807
scRNAseq hnscc Endothelial 21987
scRNAseq hnscc Erythrocyte 488
scRNAseq hnscc Malignant 71902
scRNAseq hnscc Mast 4110
scRNAseq hnscc Monocyte 33075
scRNAseq hnscc NK 10517
scRNAseq hnscc Plasma 11576
scRNAseq hnscc Treg 6607
scRNAseq mel

In [32]:
chart = alt.Chart(df.sort_values('num cells',ascending=False )).mark_circle().encode(
        x=alt.X('cell type:N'),
        y=alt.Y('disease:N'),
        size='num cells',
        color='data type')
chart

KeyError: 'num cells'

In [33]:
import seaborn as sns

In [34]:
charts = None
for i, dtype in enumerate(sorted(set(df['Data type']))):
    chart = alt.Chart(df[df['Data type']==dtype], title=dtype,
        ).mark_circle(color=sns.color_palette().as_hex()[i]).encode(
        x=alt.X('Cell type:N', axis=alt.Axis(labels=True if i==2 else False, title='Cell type' if i==2 else None)),
        y=alt.Y('Disease:N'),
        size='Number of cells')
    if charts is None:
        charts = chart
    else:
        charts = alt.vconcat(charts, chart)

charts

alt.VConcatChart(...)

#### cross-disease classification

In [22]:
DATA_DIR = './results/11302020_teir1_cross_disease/'

fps = sorted(os_helpers.listfiles(DATA_DIR, regex='.tsv$'))
# fps = [x for x in fps if 'seurat' not in x]
fps

['./results/11302020_teir1_cross_disease/scRNAseq/brca_train_brca_val/SingleCellNet.tsv',
 './results/11302020_teir1_cross_disease/scRNAseq/brca_train_brca_val/pollock.tsv',
 './results/11302020_teir1_cross_disease/scRNAseq/brca_train_brca_val/scanpy_ingest.tsv',
 './results/11302020_teir1_cross_disease/scRNAseq/brca_train_cesc_val/SingleCellNet.tsv',
 './results/11302020_teir1_cross_disease/scRNAseq/brca_train_cesc_val/pollock.tsv',
 './results/11302020_teir1_cross_disease/scRNAseq/brca_train_cesc_val/scanpy_ingest.tsv',
 './results/11302020_teir1_cross_disease/scRNAseq/brca_train_hnscc_val/SingleCellNet.tsv',
 './results/11302020_teir1_cross_disease/scRNAseq/brca_train_hnscc_val/pollock.tsv',
 './results/11302020_teir1_cross_disease/scRNAseq/brca_train_hnscc_val/scanpy_ingest.tsv',
 './results/11302020_teir1_cross_disease/scRNAseq/brca_train_melanoma_val/SingleCellNet.tsv',
 './results/11302020_teir1_cross_disease/scRNAseq/brca_train_melanoma_val/pollock.tsv',
 './results/11302020_te

In [175]:
def cap_list(ls, n=100, split=.8, oversample=True):
    """Cap list at n.

    If n is larger than list size * .8, oversample until you hit n.
    """
    cap = int(len(ls) * split)
    if cap > n:
        return random.sample(ls, n)

    if oversample:
        pool = random.sample(ls, cap) if cap else list(ls)
        ## oversample to
        return random.choices(pool, k=n)

    return random.sample(ls, cap)

In [176]:
import random
cap_list([1,2,3,4])

[3,
 1,
 1,
 3,
 2,
 3,
 1,
 3,
 3,
 2,
 1,
 1,
 1,
 2,
 3,
 1,
 1,
 2,
 1,
 2,
 1,
 3,
 3,
 3,
 3,
 3,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 3,
 3,
 3,
 3,
 2,
 2,
 1,
 1,
 1,
 1,
 2,
 1,
 3,
 3,
 3,
 1,
 2,
 2,
 3,
 2,
 1,
 3,
 2,
 3,
 3,
 1,
 2,
 3,
 3,
 3,
 1,
 3,
 2,
 1,
 1,
 2,
 2,
 2,
 1,
 3,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 3,
 1,
 3,
 3,
 1,
 1,
 3,
 2,
 3,
 1,
 3,
 3,
 3,
 2,
 2,
 3,
 1,
 2,
 1]

In [23]:

pred_map = {}
for fp in fps:
    dtype = fp.split('/')[-3]
    disease = fp.split('/')[-2].upper()
    method = fp.split('/')[-1].replace('.tsv', '')
    if 'actinn' in method: method = method.upper()
    if 'scanpy' in method: method = 'Scanpy Ingest'
    if 'pollock' in method: method = 'Pollock'
    if 'seurat_transfer' in method: method = 'Seurat Transfer'
    try:
        df = pd.read_csv(fp, sep='\t')
        df.columns = ['cell_id', 'groundtruth', 'predicted', 'probability']
        if dtype not in pred_map: pred_map[dtype] = {}
        if disease not in pred_map[dtype]: pred_map[dtype][disease] = {}
        pred_map[dtype][disease][method] = df
    except:
        print('failed: ', fp)


In [24]:
combined = None
for dtype in dtype_order:
    d = pred_map[dtype]
    df = get_performance_df(d)
    df['train_dataset'] = [x.split('_')[0] for x in df['disease']]
    df['val_dataset'] = [x.split('_')[2] for x in df['disease']]
    print(df.shape)
    chart = make_point_chart(df) + make_tick_chart(df, title=dtype)
    if combined is None:
        combined = chart
    else:
        combined |= chart
combined.resolve_scale(y='shared')

(78, 5)
(18, 5)
(43, 5)


alt.HConcatChart(...)

#### cross-disease classification

In [12]:
DATA_DIR = './results/10272020_teir1_cross_datatype/'

fps = sorted(os_helpers.listfiles(DATA_DIR, regex='.tsv$'))
# fps = [x for x in fps if 'seurat' not in x]
fps

['./results/10272020_teir1_cross_datatype/snRNAseq/scRNAseq_brca_train_snATACseq_brca_val/SingleCellNet.tsv',
 './results/10272020_teir1_cross_datatype/snRNAseq/scRNAseq_brca_train_snATACseq_brca_val/pollock.tsv',
 './results/10272020_teir1_cross_datatype/snRNAseq/scRNAseq_brca_train_snATACseq_brca_val/scanpy_ingest.tsv',
 './results/10272020_teir1_cross_datatype/snRNAseq/scRNAseq_brca_train_snATACseq_brca_val/seurat_transfer.tsv',
 './results/10272020_teir1_cross_datatype/snRNAseq/scRNAseq_brca_train_snATACseq_gbm_val/SingleCellNet.tsv',
 './results/10272020_teir1_cross_datatype/snRNAseq/scRNAseq_brca_train_snATACseq_gbm_val/pollock.tsv',
 './results/10272020_teir1_cross_datatype/snRNAseq/scRNAseq_brca_train_snATACseq_gbm_val/scanpy_ingest.tsv',
 './results/10272020_teir1_cross_datatype/snRNAseq/scRNAseq_brca_train_snATACseq_gbm_val/seurat_transfer.tsv',
 './results/10272020_teir1_cross_datatype/snRNAseq/scRNAseq_brca_train_snRNAseq_brca_val/SingleCellNet.tsv',
 './results/10272020_te

In [13]:

pred_map = {}
for fp in fps:
    dtype = fp.split('/')[-3]
    disease = fp.split('/')[-2].upper()
    method = fp.split('/')[-1].replace('.tsv', '')
    if 'actinn' in method: method = method.upper()
    if 'scanpy' in method: method = 'Scanpy Ingest'
    if 'pollock' in method: method = 'Pollock'
    if 'seurat_transfer' in method: method = 'Seurat Transfer'
    try:
        df = pd.read_csv(fp, sep='\t')
        df.columns = ['cell_id', 'groundtruth', 'predicted', 'probability']
        if dtype not in pred_map: pred_map[dtype] = {}
        if disease not in pred_map[dtype]: pred_map[dtype][disease] = {}
        pred_map[dtype][disease][method] = df
    except:
        print('failed: ', fp)


failed:  ./results/10272020_teir1_cross_datatype/snRNAseq/scRNAseq_brca_train_snATACseq_brca_val/seurat_transfer.tsv
failed:  ./results/10272020_teir1_cross_datatype/snRNAseq/scRNAseq_brca_train_snATACseq_gbm_val/seurat_transfer.tsv
failed:  ./results/10272020_teir1_cross_datatype/snRNAseq/scRNAseq_cesc_train_snATACseq_brca_val/seurat_transfer.tsv
failed:  ./results/10272020_teir1_cross_datatype/snRNAseq/scRNAseq_cesc_train_snATACseq_gbm_val/seurat_transfer.tsv
failed:  ./results/10272020_teir1_cross_datatype/snRNAseq/scRNAseq_hnscc_train_snATACseq_brca_val/seurat_transfer.tsv
failed:  ./results/10272020_teir1_cross_datatype/snRNAseq/scRNAseq_hnscc_train_snATACseq_gbm_val/seurat_transfer.tsv
failed:  ./results/10272020_teir1_cross_datatype/snRNAseq/scRNAseq_melanoma_train_snATACseq_brca_val/seurat_transfer.tsv
failed:  ./results/10272020_teir1_cross_datatype/snRNAseq/scRNAseq_melanoma_train_snATACseq_gbm_val/seurat_transfer.tsv
failed:  ./results/10272020_teir1_cross_datatype/snRNAseq/

In [15]:
combined = None
for dtype in ['snRNAseq']:
    d = pred_map[dtype]
    df = get_performance_df(d)
    df['train_dataset'] = [x.split('_')[0] for x in df['disease']]
    df['val_dataset'] = [x.split('_')[2] for x in df['disease']]
    print(df.shape)
    chart = make_point_chart(df) + make_tick_chart(df, title=dtype)
    if combined is None:
        combined = chart
    else:
        combined |= chart
combined.resolve_scale(y='shared')

(216, 5)


alt.LayerChart(...)

In [3]:
import scanpy as sc

###### sandbox

In [ ]:
fps = sorted(os_helpers.listfiles('./data/01272021_harmonized/teir_1/', regex='.h5ad$'))
fps = [x for x in fps if 'train' not in x and 'val' not in x and 'gene' not in x and 'motif' not in x and 'peak' not in x]
fps

In [4]:
a1 = sc.read_h5ad('./data/01272021_harmonized/teir_1/scRNAseq/melanoma.h5ad')
a2 = sc.read_h5ad('./data/01272021_harmonized/teir_1/scRNAseq/myeloma.h5ad')

In [9]:
a3 = sc.read_h5ad('./data/01272021_harmonized/teir_1/scRNAseq/brca.h5ad')

In [6]:
a1.var

,detection_rate,gmean,variance,residual_mean,residual_variance
AL627309.1,0.001024,0.000710,0.001023,0.005429,1.205686
LINC01409,0.020497,0.014736,0.023200,0.007018,1.024149
FAM87B,0.000967,0.000699,0.001231,-0.000056,0.995660
LINC00115,0.011794,0.008482,0.014008,0.009830,1.207445
FAM41C,0.021824,0.015749,0.025406,-0.000743,0.930020
...,...,...,...,...,...
AL354822.1,0.008153,0.005783,0.008936,-0.018245,0.514212
LOC107987373,0.061454,0.058716,0.245925,-0.043126,0.742454
AC233755.2,0.000152,0.000121,0.000265,0.004714,1.940715
LOC102723407,0.000114,0.000092,0.000265,-0.003213,0.303124


In [7]:
a2.var

""
index
FO538757.2
AP006222.2
RP11-206L10.9
LINC00115
FAM41C
...
TEK
RP11-800A3.7
ENO4


In [10]:
len(set(a1.var.index).intersection(set(a3.var.index)))

21535